In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
gcp_username = 'tejas'

In [3]:
num_nodes = 4

In [4]:
experiment = 'set_'+str(num_nodes)+'nodes'

In [5]:

commands  = []



for i in range(num_nodes):
    cmd =  'gcloud compute instances create nuraft-'+f"{i:03}"+' --project=ucr-ursa-major-lesani-lab \
    --zone=us-central1-c \
    --machine-type=e2-highcpu-4 \
    --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
    --can-ip-forward \
    --maintenance-policy=MIGRATE \
    --provisioning-model=STANDARD \
    --service-account=961693926925-compute@developer.gserviceaccount.com \
    --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
    --tags=http-server,https-server \
    --create-disk=auto-delete=yes,boot=yes,device-name=instance-20250611-154850,disk-resource-policy=projects/ucr-ursa-major-lesani-lab/regions/us-central1/resourcePolicies/default-schedule-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-minimal-2504-plucky-amd64-v20250606,mode=rw,size=10,type=pd-balanced \
    --no-shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --labels=goog-ec-src=vm_add-gcloud \
    --reservation-affinity=any'

    commands.append(cmd)


def run_command(command):
    subprocess.call(command, shell=True)


# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-000].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-002].


NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP    EXTERNAL_IP     STATUS
nuraft-000  us-central1-c  e2-highcpu-4               10.128.15.233  34.123.137.203  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-001].


NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
nuraft-002  us-central1-c  e2-highcpu-4               10.128.0.44  34.41.232.84  RUNNING
NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
nuraft-001  us-central1-c  e2-highcpu-4               10.128.0.58  34.134.156.149  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-003].


NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
nuraft-003  us-central1-c  e2-highcpu-4               10.128.0.48  35.222.80.144  RUNNING


In [6]:
redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' gcp_ips.txt')

0

In [7]:
f = open('gcp_ips.txt', "r")
iplist = [line.strip() for line in f.readlines()]

In [8]:
iplist

['10.128.15.233', '10.128.0.58', '10.128.0.44', '10.128.0.48']

In [12]:
def setup_remote(i):


    command = 'gcloud compute ssh --zone "'+'us-central1-c'+'" "nuraft-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "\
    sudo apt update; \
    sudo apt -y install build-essential git cmake openssl libssl-dev libz-dev pkg-config libzstd-dev screen psmisc > setup.log;\
    git clone https://github.com/tejas-shivanand-mane/NuRaft.git"'

    print(command)

    output = os.system(command)
    print(output)


In [13]:
results = Parallel(n_jobs=60)(delayed(setup_remote)(i) for i in range(len(iplist)) )


print(results)  

Hit:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease
Hit:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease
Hit:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu plucky-security InRelease
Reading package lists...Hit:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease
Hit:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease
Hit:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease


Hit:4 http://security.ubuntu.com/ubuntu plucky-security InRelease
Hit:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease
Hit:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease
Hit:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease
Reading package lists...Hit:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease
Hit:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease
Hit:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu plucky-security InRelease
Reading package lists...Hit:4 http://security.ubuntu.com/ubuntu plucky-security InRelease
Reading package lists...
Building dependency tree...
Building dependency tree...
Reading state information...
13 packages can be upgraded. Run 'apt list --upgradable' to see them.

Building dependency tree...
Reading state information...


13 packages can be upgraded. Run 'apt list --upgradable' to see them.

Reading state information...
13 packages can be upgraded. Run 'apt list --upgradable' to see them.

Building dependency tree...


Reading state information...
13 packages can be upgraded. Run 'apt list --upgradable' to see them.




fatal: destination path 'NuRaft' already exists and is not an empty directory.
fatal: destination path 'NuRaft' already exists and is not an empty directory.
fatal: destination path 'NuRaft' already exists and is not an empty directory.


[None, None, None, None]


fatal: destination path 'NuRaft' already exists and is not an empty directory.


In [14]:

def compile_remote(i):

  


    command = 'gcloud compute ssh --zone "'+'us-central1-c'+'" "nuraft-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "\
    cd NuRaft; \
    ./prepare.sh; cp CMakeListsUbuntu.txt CMakeLists.txt;\
    mkdir build; cd build; cmake .. ; make -j4;"'

    print(command)

    output = os.system(command)
    print(output)



In [ ]:
results = Parallel(n_jobs=60)(delayed(compile_remote)(i) for i in range(len(iplist)) )


print(results)  

+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
Cloning into './asio'...
+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
Cloning into './asio'...
+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
Cloning into './asio'...
+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
Cloning into './asio'...
+ cd asio
+ git checkout asio-1-24-0
+ cd asio
+ git checkout asio-1-24-0
+ cd asio
+ git checkout asio-1-24-0
Note:

-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done


Note: switching to 'asio-1-24-0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 147f7225a asio version 1.24.0 released
+ cd ..


-- Found PkgConfig: /usr/bin/pkg-config (found version "1.8.1")
-- Checking for module 'libzstd'
-- Found PkgConfig: /usr/bin/pkg-config (found version "1.8.1")
-- Checking for module 'libzstd'
--   Found libzstd, version 1.5.6
--   Found libzstd, version 1.5.6
-- Detecting CXX compiler ABI info - done
-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- C++ Standard: 11
-- Build type: RelWithDebInfo
-- Build Install Prefix : /usr/local
-- C++ Standard: 11
-- Build type: RelWithDebInfo
-- Build Install Prefix : /usr/local
-- Found PkgConfig: /usr/bin/pkg-config (found version "1.8.1")
-- Checking for module 'libzstd'
--   Found libzstd, version 1.5.6
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.a (found version "1.3.1")
-- C++ Standard: 11
-- Build type: RelWithDebInfo
-- Build Install Prefix : /usr/local
-- Performing T

/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘void nuraft::raft_server::handle_peer_resp(nuraft::ptr<nuraft::resp_msg>&, nuraft::ptr<nuraft::rpc_exception>&)’:
/home/tejas/NuRaft/src/raft_server.cxx:840:28: warning: unused variable ‘rc’ [-Wunused-variable]
  840 |                 const auto rc = ctx_->cb_func_.call(cb_func::FollowerLost, &param);
      |                            ^~
/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘bool nuraft::raft_server::check_leadership_validity()’:
/home/tejas/NuRaft/src/raft_server.cxx:1159:24: warning: unused variable ‘rc’ [-Wunused-variable]
 1159 |             const auto rc = ctx_->cb_func_.call(cb_func::FollowerLost, &param);
      |                        ^~


...

[ 23%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/stat_mgr.cxx.o
[ 23%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/stat_mgr.cxx.o
[ 21%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/snapshot_sync_req.cxx.o
[ 22%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/srv_config.cxx.o
[ 23%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/stat_mgr.cxx.o
[ 23%] Built target RAFT_CORE_OBJ
[ 24%] Linking CXX static library libnuraft.a
[ 25%] Linking CXX shared library libnuraft.so
[ 23%] Built target RAFT_CORE_OBJ
[ 25%] Linking CXX static library libnuraft.a
[ 25%] Linking CXX shared library libnuraft.so
[ 25%] Built target static_lib
[ 23%] Built target RAFT_CORE_OBJ
[ 25%] Linking CXX static library libnuraft.a
[ 25%] Linking CXX shared library libnuraft.so
[ 26%] Building CXX object examples/CMakeFiles/stack_server.dir/stack_ops/stack_server.cxx.o
[ 28%] Building CXX object examples/CMakeFiles/calc_server.dir/calculator/calc_server.cxx.o
[ 28%] Building C

In file included from /home/tejas/NuRaft/examples/set_ops/set_server.cxx:18:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> set_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx:67:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   67 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~
In file included from /home/tejas/NuRaft/examples/stack_ops/stack_server.cxx:18:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> stack_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx:68:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   68 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~
In file included from /home/tejas/NuRa

[ 23%] Built target RAFT_CORE_OBJ
[ 25%] Linking CXX shared library libnuraft.so
[ 25%] Linking CXX static library libnuraft.a
[ 25%] Built target static_lib
[ 27%] Building CXX object examples/CMakeFiles/calc_server.dir/calculator/calc_server.cxx.o
[ 27%] Building CXX object examples/CMakeFiles/stack_server.dir/stack_ops/stack_server.cxx.o
[ 28%] Building CXX object examples/CMakeFiles/set_server.dir/set_ops/set_server.cxx.o
[ 28%] Built target shared_lib
[ 29%] Building CXX object examples/CMakeFiles/stack_server.dir/logger.cc.o
[ 29%] Building CXX object examples/CMakeFiles/calc_server.dir/logger.cc.o
[ 29%] Building CXX object examples/CMakeFiles/stack_server.dir/logger.cc.o
[ 29%] Building CXX object examples/CMakeFiles/calc_server.dir/logger.cc.o


In file included from /home/tejas/NuRaft/examples/set_ops/set_server.cxx:18:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> set_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx:67:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   67 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~
In file included from /home/tejas/NuRaft/examples/stack_ops/stack_server.cxx:18:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> stack_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx:68:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   68 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~


[ 30%] Building CXX object examples/CMakeFiles/stack_server.dir/in_memory_log_store.cxx.o
[ 31%] Building CXX object examples/CMakeFiles/set_server.dir/logger.cc.o
[ 30%] Building CXX object examples/CMakeFiles/stack_server.dir/in_memory_log_store.cxx.o
[ 31%] Building CXX object examples/CMakeFiles/set_server.dir/logger.cc.o
[ 30%] Building CXX object examples/CMakeFiles/set_server.dir/in_memory_log_store.cxx.o
[ 31%] Building CXX object examples/CMakeFiles/stack_server.dir/logger.cc.o
[ 32%] Building CXX object examples/CMakeFiles/set_server.dir/in_memory_log_store.cxx.o
[ 32%] Building CXX object examples/CMakeFiles/calc_server.dir/in_memory_log_store.cxx.o
[ 32%] Building CXX object examples/CMakeFiles/stack_server.dir/in_memory_log_store.cxx.o
[ 33%] Building CXX object examples/CMakeFiles/calc_server.dir/in_memory_log_store.cxx.o
[ 33%] Building CXX object examples/CMakeFiles/set_server.dir/in_memory_log_store.cxx.o
[ 29%] Building CXX object examples/CMakeFiles/calc_server.dir/l

In [ ]:
os.system('git add .; git commit -m "test"; git push -f origin HEAD:master')

In [ ]:

def git_pull(i):

  


    command = 'gcloud compute ssh --zone "'+'us-central1-c'+'" "nuraft-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "\
    cd NuRaft; \
    git pull;"'

    print(command)

    output = os.system(command)
    print(output)

results = Parallel(n_jobs=60)(delayed(git_pull)(i) for i in range(len(iplist)) )


In [ ]:
results = Parallel(n_jobs=60)(delayed(compile_remote)(i) for i in range(len(iplist)) )


print(results)  

In [ ]:
def run_project_server(i):
    instance = f"nuraft-{i:03}"
    ip = iplist[i]
    session_name = f"server{i}"
    log_file = f"/home/tejas/NuRaft/build/examples/set_server{i}.log"
    command = f'''
    gcloud compute ssh --zone "us-central1-c" "{instance}" --project "ucr-ursa-major-lesani-lab" --command "\
    cd NuRaft/build/examples; \
    screen -dmS {session_name} bash -c './set_server {i+1} {ip}:10001 > {log_file} 2>&1'"
    '''
    print(f"Launching on instance {instance}")
    output = os.system(command)
    print(f"Return code: {output}")




# Run in parallel across all IPs
# results = Parallel(n_jobs=60)(delayed(run_project_server)(i) for i in [0,1,2])


In [ ]:
# results = Parallel(n_jobs=60)(delayed(run_project_server)(i) for i in [0])
run_project_server(2)
run_project_server(1)



In [ ]:
run_project_server(0)


In [ ]:
def kill_project_server(i):
    instance = f"nuraft-{i:03}"
    session_name = f"server{i}"
    command = f'''
    gcloud compute ssh --zone "us-central1-c" "{instance}" --project "ucr-ursa-major-lesani-lab" --command "\
    screen -S {session_name} -X quit"
    '''
    print(f"Killing server on instance {instance}")
    output = os.system(command)
    print(f"Return code: {output}")

results = Parallel(n_jobs=60)(delayed(kill_project_server)(i) for i in range(len(iplist)) )


In [ ]:
def copy_logs_to_local(i, local_dir="../nuraft_logs/"+experiment):

    if not os.path.exists("../nuraft_logs/"+experiment):
        os.mkdir("../nuraft_logs/"+experiment)
        
    instance = f"nuraft-{i:03}"
    remote_path = f"/home/tejas/NuRaft/build/examples/set_server{i}.log"
    local_path = f"{local_dir}/set_server{i}.log"
    
    # Make sure local directory exists
    os.makedirs(local_dir, exist_ok=True)
    
    command = f'gcloud compute scp --zone "us-central1-c" "{instance}:{remote_path}" "{local_path}" --project "ucr-ursa-major-lesani-lab"'
    
    print(f"Copying log from {instance} to {local_path} ...")
    ret = os.system(command)
    print(f"Return code: {ret}")

# Example usage: copy logs for servers 0, 1, 2
for i in range(3):
    copy_logs_to_local(i)

In [ ]:

os.system('gcloud compute instances delete --zone=us-central1-c --quiet $(gcloud compute instances list --format="value(name)")')
